In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# print(os.environ['LD_PRELOAD'])
# os.environ['LD_PRELOAD'] = '/home/ubuntu/slime/working/.venv/lib/python3.10/site-packages/torch_memory_saver_hook_mode_preload.abi3.so'
# os.environ["TMS_INIT_ENABLE"] = ""
# os.environ["TMS_INIT_ENABLE_CPU_BACKUP"] = ""


In [ ]:
from torch_memory_saver import configure_subprocess, torch_memory_saver
with configure_subprocess():
  torch_memory_saver._ensure_initialized()

In [3]:
import torch


def get_and_print_gpu_memory(message, gpu_id=0):
    """Print GPU memory usage with optional message"""
    mem = torch.cuda.device_memory_used(gpu_id)
    print(f"GPU {gpu_id} memory: {mem / 1024 ** 3:.2f} GB ({message})")
    return mem

# Print devices
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))
print(get_and_print_gpu_memory("before"))


1
0
NVIDIA A100-SXM4-80GB
GPU 0 memory: 0.75 GB (before)
806879232


In [4]:
from torch_memory_saver import torch_memory_saver
torch_memory_saver.hook_mode = "torch"
import torch
with torch_memory_saver.region():
  tensor1 = torch.full((5_000_000_000,), 100, dtype=torch.uint8, device='cuda')
get_and_print_gpu_memory("after_alloc")
torch_memory_saver.pause()

get_and_print_gpu_memory("after_pause")
torch_memory_saver.resume()
get_and_print_gpu_memory("after_resume")


GPU 0 memory: 5.89 GB (after_alloc)
GPU 0 memory: 1.24 GB (after_pause)
GPU 0 memory: 5.89 GB (after_resume)


6328221696

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt


In [ ]:
from slime.utils.memory_utils import available_memory
available_memory()

In [ ]:
# print("Loading veomni_first_sample.pt...")
# data = torch.load("veomni_first_sample.pt")

# print(f"Data keys: {data.keys()}")
# print(f"Tokens length: {len(data['tokens'])}")
# print(f"Log probs length: {len(data['log_probs'])}")

# tokens = data["tokens"]
# reference_log_probs = data["log_probs"]

# Convert tensors to lists if needed
# if isinstance(tokens, torch.Tensor):
#     tokens = tokens.tolist()
# if isinstance(reference_log_probs, torch.Tensor):
#     reference_log_probs = reference_log_probs.tolist()

# print(f"Tokens type: {type(tokens[0])}")
# print(f"First few tokens: {tokens[:10]}")

# Load Qwen model and tokenizer
print("Loading Qwen/Qwen3-0.6B model and tokenizer...")
# model_name = "Qwen/Qwen3-0.6B"
model_name = "Qwen/Qwen3-4B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, attn_implementation="flash_attention_2", dtype="bfloat16", device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_name,  dtype="bfloat16", device_map="auto")

# Convert tokens to text
# text = tokenizer.decode(tokens)
# print(f"Decoded text length: {len(text)} characters")

# Re-tokenize to get proper tokenization
# tokenized = tokenizer(text, return_tensors="pt")
# input_ids = torch.tensor(tokens).reshape(1, -1).to(model.device, dtype=torch.int64)
# print(f"Input ids shape: {input_ids.shape}")
# # Get logits from model
# with torch.no_grad():
#     outputs = model(input_ids)
#     logits = outputs.logits

In [ ]:
from torch_memory_saver import torch_memory_saver
from slime.utils.memory_utils import clear_memory

print("Pausing memory saver")
print(available_memory())
torch_memory_saver.pause()
clear_memory()
print("After pause")
print(available_memory())

In [ ]:
from torch_memory_saver import torch_memory_saver

print("resuming memory saver")
print(available_memory())
torch_memory_saver.resume()
print("After resume")
print(available_memory())

In [ ]:
from slime.utils.memory_utils import clear_memory
# del model
clear_memory()
print(available_memory())

In [ ]:
log_probs = torch.log_softmax(logits, dim=-1)
print(f"Log probs shape: {log_probs.shape}")

# Extract log probabilities for the actual tokens (excluding the last token)
# We need log_probs[i] for token[i+1]
model_log_probs = []
for i in range(len(tokens) - 1):
    token_id = tokens[i + 1]
    log_prob = log_probs[0, i, token_id].item()
    model_log_probs.append(log_prob)

In [ ]:
model_log_probs = torch.tensor(model_log_probs, device=torch.device("cuda")).bfloat16()
model_log_probs

In [ ]:
# l2 norm between model_log_probs and reference_log_probs

torch.norm(model_log_probs - reference_log_probs) / len(model_log_probs)


In [ ]:
samples = torch.load("./test/debug_rollout_data")


In [ ]:
megatron_data = torch.load(open("./megatron_rollout_data_0.pt", "rb"))
veomni_data = torch.load(open("./veomni_padded_batches_0.pt", "rb"))
fsdp_data = torch.load(open("./fsdp_padded_batches_0.pt", "rb"))

# megatron_data["abs_advantages"] = [k.abs() for k in megatron_data["advantages"]]
# megatron_data["abs_advantages"] = [k.abs() for k in megatron_data["advantages"]]
# print(megatron_data, veomni_data)


In [ ]:
# len(veomni_data)
print(veomni_data[0])
print(len(veomni_data[0]["tokens"][0]))
print(len(veomni_data[0]["log_probs"][0]))
print(len(fsdp_data[0]["log_probs"][0]))
print(veomni_data[0]["loss_masks"].sum())


In [ ]:
megatron_first_sample_tokens = megatron_data["tokens"][0]
veomni_first_sample_tokens = veomni_data[0]["tokens"][0]
first_samples_tokens = samples['samples'][0]['tokens']
second_samples_tokens = samples['samples'][1]['tokens']

# Convert all to int32 cuda tensors
megatron_first_sample_tokens = torch.tensor(megatron_first_sample_tokens, device=torch.device("cuda")).int()
veomni_first_sample_tokens = torch.tensor(veomni_first_sample_tokens, device=torch.device("cuda")).int()
first_samples_tokens = torch.tensor(first_samples_tokens, device=torch.device("cuda")).int()
second_samples_tokens = torch.tensor(second_samples_tokens, device=torch.device("cuda")).int()

# Assert shpaes are equal for first samples
assert megatron_first_sample_tokens.shape == veomni_first_sample_tokens.shape == first_samples_tokens.shape, f"{megatron_first_sample_tokens.shape} != {veomni_first_sample_tokens.shape} != {first_samples_tokens.shape}"

print(f"shape: {megatron_first_sample_tokens.shape}")
assert torch.allclose(megatron_first_sample_tokens, veomni_first_sample_tokens)
assert torch.allclose(first_samples_tokens, megatron_first_sample_tokens)
assert not torch.allclose(torch.tensor(second_samples_tokens, device=torch.device("cuda")), torch.tensor(megatron_first_sample_tokens, device=torch.device("cuda")))
# print(veomni_first_sample_tokens[0:20])

print(megatron_first_sample_tokens[-20:])
data = {
  'tokens': veomni_first_sample_tokens,
  'log_probs': veomni_data[0]["log_probs"][0],
  'response_lengths': len(megatron_data["log_probs"][0])
}
print(data)

torch.save(data, "veomni_first_sample.pt")
# megatron_first_sample_tokens[0:20]

In [ ]:
megatron_lprobs = megatron_data["log_probs"][0]
veomni_lprobs = veomni_data[0]["log_probs"][0]
fsdp_lprobs = fsdp_data[0]["log_probs"][0]
sample_lprobs = samples['samples'][0]['rollout_log_probs']

# megatron_lprobs
# Convert all to bf16 cuda tensors  
megatron_lprobs = torch.tensor(megatron_lprobs, device=torch.device("cuda")).bfloat16()
veomni_lprobs = torch.tensor(veomni_lprobs, device=torch.device("cuda")).bfloat16()
sample_lprobs = torch.tensor(sample_lprobs, device=torch.device("cuda")).bfloat16()
fsdp_lprobs = torch.tensor(fsdp_lprobs, device=torch.device("cuda")).bfloat16()
# Print lengths
print(len(megatron_lprobs))
print(len(veomni_lprobs))
print(len(sample_lprobs))
veomni_lprobs = veomni_lprobs[-len(sample_lprobs):]
fsdp_lprobs = fsdp_lprobs[-len(sample_lprobs):]
print(len(veomni_lprobs))
print(veomni_lprobs)
# assert torch.allclose(megatron_lprobs, veomni_lprobs)
# assert torch.allclose(sample_lprobs, megatron_lprobs)



In [ ]:
print(megatron_lprobs[:5])
print(veomni_lprobs[:5])
print(fsdp_lprobs[:5])
print(sample_lprobs[:5])

In [ ]:
print(torch.allclose(megatron_lprobs, sample_lprobs, atol=1e-1))
print(torch.allclose(veomni_lprobs, sample_lprobs, atol=1e-1))
print(torch.allclose(megatron_lprobs, veomni_lprobs, atol=1e-1))



In [ ]:
# print l2 norm of the difference between megatron_lprobs and veomni_lprobs
print(torch.norm(megatron_lprobs - veomni_lprobs)/len(megatron_lprobs))
print(torch.norm(megatron_lprobs - sample_lprobs)/len(megatron_lprobs))
print(torch.norm(veomni_lprobs - sample_lprobs)/len(veomni_lprobs))
print(torch.norm(fsdp_lprobs - veomni_lprobs)/len(fsdp_lprobs))

In [ ]:
# print max of the difference between megatron_lprobs and veomni_lprobs
print(torch.max(torch.abs(megatron_lprobs - veomni_lprobs)))
print(torch.max(torch.abs(megatron_lprobs - sample_lprobs)))
print(torch.max(torch.abs(veomni_lprobs - sample_lprobs)))
